In [1]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import pymongo
import warnings
import re
warnings.filterwarnings("ignore")
username = "tagger-admin"
password = "tvaiadmin"
db_client = pymongo.MongoClient('104.198.62.226', username=username,
                                password=password,
                                authSource='tags',
                                authMechanism='SCRAM-SHA-256', port=27017).tags

In [2]:
import aerospike
config = {
    'hosts': [('35.228.136.58', 3000)]
}
client = aerospike.client(config).connect()
data_as_key = ("ids_to_validate", "data", "data")
tags_dictionary = client.get(data_as_key)[-1]['tags']
len(tags_dictionary)

7737

In [3]:
bert_data_key = ("ids_to_validate", "data", "bert_tags")

tags_dictionary = client.get(data_as_key)[-1]['tags']
guten_tag_labels = client.get(bert_data_key)[-1]["gutenTag"]
categories_tag_labels = client.get(bert_data_key)[-1]["categories_tags"]

In [ ]:
def dict_values(tags_list):
    new_tags = []
    tags_collector = []
    for tag in tags_list:
        if tag.lower() in tags_dictionary:
            if tags_dictionary[tag.lower()] != [0]:
                tags_collector += tags_dictionary[tag.lower()]     
        else:
            new_tags.append(tag.lower())
            tags_collector.append(tag.lower())
    return set(tags_collector), set(new_tags)

In [ ]:
# exxon добавить энерджи
# проверить что есть в гутентаге: 'geopolitics', 'sustainability', 'digital', 'services'
#     venture, debt', 'consumer'
#     'debt', 'consumer'}
#     software

In [ ]:
db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},
                                              {"status":"graphicone_feed"}],
                                              "source": "def"
                                                   }).count()

In [3]:
db_client["tweets_pipeline_v2"].find({"$or": [{"status":"validated"}],
                                              "source": "def"
                                                   }).count()

555

In [4]:
db_client["tweets_pipeline_v2"].find({"source": "dailyshot"}).count()

1218

In [11]:
db_client["tweets_pipeline_v2"].find({"source": "twitter", "username": "soberlook"}).count()

244

In [5]:
start_date = datetime.today()
end_date = datetime(2020, 6, 18)
for i in db_client["tweets_pipeline_v2"].find({"$or": [{"status":"graphicone_search"},
                                                           {"status":"graphicone_feed"}],
                                                   'approved_timestamp': {'$gte': end_date, '$lte': start_date},
                                                   }).sort('approved_timestamp',pymongo.DESCENDING):
        
        analyst = set()
        guten_tag = set()
        entity = set()
        tagger = set()
        tagger_after_dict = set()
        new_tags_tagger = set()
        
        if "bert_tags" in i:
            guten_tag = set(i["bert_tags"])
            
        if "entity_prepared" in i:            
            try:
                entity = set(i["entity_prepared"])
            except Exception as err:
                pass
            
        if "confirmed_after_validate" in i:
            tagger = set(i["confirmed_after_validate"])
            tagger_after_dict, new_tags_tagger = dict_values(tagger)

            
        if "grafeed_confirmed" in i:
            analyst = set(i["grafeed_confirmed"])
            analyst_after_dict, new_tags_analyst = dict_values(analyst)
        
        print(i["_id"])
        print("models", list(guten_tag)+list(entity))        
        print("analyst:",analyst)        
        print("dif", set(analyst) - set(list(guten_tag)+list(entity)))
        print()
        print()

5e609f53a0e804932628d5bc
models ['ecology', 'emissions', 'environment']
analyst: {'ecology', 'plastics', 'environment', 'marine'}
dif {'plastics', 'marine'}


5e603a97ca7d7fb28f1cc58f
models ['transportation', 'vehicles', 'technology', 'global']
analyst: {'technology', 'automotive', 'telecom', 'logistics', 'transportation', 'smart city', 'innovation', 'iot', 'data', 'vehicles', 'global', 'wearables'}
dif {'automotive', 'telecom', 'logistics', 'smart city', 'innovation', 'iot', 'data', 'wearables'}


5e412d89eb8f57d996223f38
models ['wind power', 'hydro', 'renewables', 'oil', 'gas', 'nuclear power', 'energy', 'commodities', 'electricity', 'solar energy', 'new york', 'coal', 'commodities']
analyst: {'coal', 'hydro', 'renewables', 'oil', 'gas', 'nuclear power', 'new york', 'energy', 'commodities', 'electricity'}
dif set()


5ed8849ec699e47fb0029213
models ['boomers', 'generation x', 'generations', 'millennials', 'gentex', 'us']
analyst: {'social', 'generations', 'millennials', 'wealth', '

5eac5bde252baaecab1b35f8
models ['gdp', 'macro', 'deficit']
analyst: {'gdp', 'eu', 'government', 'macro', 'deficit'}
dif {'government', 'eu'}


5ee55970339829b88d33c8ef
models ['ecology', 'fuel', 'transportation', 'environment', 'emissions', 'energy', 'electricity', 'germany']
analyst: {'electric vehicles', 'ecology', 'automotive', 'transportation', 'coronavirus', 'environment', 'emissions', 'pandemic', 'vehicles', 'disease', 'energy', 'germany', 'covid-19', 'construction & engineering', 'electricity'}
dif {'electric vehicles', 'automotive', 'pandemic', 'vehicles', 'covid-19', 'construction & engineering', 'disease', 'coronavirus'}


5ee57dac65b988394377dc33
models ['electric vehicles', 'fuel', 'commodities', 'gasoline', 'commodities']
analyst: {'electric vehicles', 'gasoline', 'automotive', 'fuel', 'energy', 'commodities', 'vehicles', 'diesel'}
dif {'vehicles', 'automotive', 'diesel', 'energy'}


5ee569a487081ad9e8a255b9
models ['large caps', 'index', 's&p']
analyst: {'index', 'large 

models ['leisure', 'retail', 'consumer', 'services', 'transportation', 'consumer staples', 'inflation', 'utilities', 'education', 'health', 'restaurants', 'real estate', 'alcohol', 'consumer discretionary']
analyst: {'macro', 'utilities', 'index', 'eu', 'water', 'consumer staples', 'restaurants', 'health', 'real estate', 'energy', 'travel', 'cpi', 'retail', 'education', 'alcohol', 'leisure', 'telecom', 'services', 'household', 'transportation', 'home & decor', 'inflation', 'culture'}
dif {'index', 'telecom', 'household', 'eu', 'home & decor', 'macro', 'water', 'travel', 'culture', 'energy', 'cpi'}


5defdc51c1bd5a6cafc91f8d
models ['leisure', 'retail', 'services', 'transportation', 'utilities', 'education', 'health', 'restaurants', 'real estate', 'commodities', 'alcohol', 'tobacco']
analyst: {'fuel', 'macro', 'utilities', 'index', 'eu', 'apparel', 'water', 'gas', 'restaurants', 'health', 'real estate', 'energy', 'cpi', 'travel', 'retail', 'education', 'commodities', 'alcohol', 'tobacco